<center> <h1>Simulador de sistemas solares </h1> </center> 

In [1]:
%%capture
# Download latest FFmpeg static build.  
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

In [11]:
# importa librerías necesarias para la simulación
from IPython.display import HTML, Javascript, display
import ipywidgets as widgets
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from astropy.time import Time
from astroquery.jplhorizons import Horizons
from ipywidgets import *

In [12]:
sim_start_date = "2019-01-01" # fecha inicial de la simulación (año-mes-día)
sim_end_date = "2020-12-31" # fecha final de la simulación (año-mes-día)
diff = Time(sim_end_date)-Time(sim_start_date)
sim_duration = int(diff.value) # duración de la simulación en días

In [13]:
# Clase de los objectos a incluir en la simulación (sol y planetas)
class Object:                  
    def __init__(self, name, rad, color, r, v):
        self.name = name # nombre
        self.r    = np.array(r, dtype=np.float) # posición
        self.v    = np.array(v, dtype=np.float) # velocidad
        self.xs = [] # posición eje x
        self.ys = [] # posición eje y
        self.plot = ax.scatter(r[0], r[1], color=color, s=rad**2, edgecolors=None, zorder=10) # gráfico del objeto
        self.line, = ax.plot([], [], color=color, linewidth=1.4) # gráfico de la órbita

In [14]:
# Clase del sistema solar a simular
class SolarSystem:
    def __init__(self, thesun):
        self.thesun = thesun # sol
        self.planets = [] # planetas
        self.time = None # tiempo
        # texto de la fecha
        self.timestamp = ax.text(.03, .94, 'Fecha: ', color='w', transform=ax.transAxes, fontsize='x-large')
    def add_planet(self, planet): # método para añadir planetas a la simulación
        self.planets.append(planet)
    def evolve(self): # método para calcular la trayectoria de un planeta
        # texto de la fecha    
        time_str = Time(self.time, format='jd').strftime('%d-%m-%Y')
        self.timestamp.set_text('Fecha: ' + time_str)
        dates = [self.timestamp]
        dt = 1.0 # intervalo de tiempo mínimo (un día)
        self.time += 1 # fecha
        plots = []
        lines = []
        for p in self.planets: # cada planeta pasa por la siguiente rutina
            p.r += p.v * dt # posición 
            acc = -2.959e-4 * p.r / np.sum(p.r**2)**(3./2) # aceleración (en UA/día^2)
            p.v += acc * dt # velocidad 
            # se guardan las componentes x e y de la posición en listas
            p.xs.append(p.r[0]) 
            p.ys.append(p.r[1]) 
            p.plot.set_offsets(p.r[:2]) # animación del movimiento 
            # se grafica la órbita
            p.line.set_xdata(p.xs)
            p.line.set_ydata(p.ys)
            # se guardan los gráficos en listas
            plots.append(p.plot) 
            lines.append(p.line) 
        return plots + lines + dates # genera gráficos de la simulación 

In [15]:
# Función para simular la evolución del sistema solar
def animate(i):
    return ss.evolve()

In [16]:
# menú para seleccionar la cantidad de planetas a incluir 
numplane = widgets.Dropdown(options=[0,1,2,3,4], value=4, description='Planetas', disabled=False)

In [17]:
npln = numplane.value # número de planetas definido por el usuario
s, c, t = [[] for n in range(3)] # listas para los valores de las propiedades de los planetas
# valores por defecto de las propiedades 
names = ['Mercurio', 'Venus', 'Tierra', 'Marte'] # nombres
colors = ['gray', 'orange', 'blue', 'chocolate'] # colores 
sizes = [0.38, 0.95, 1., 0.53] # tamaños en relación con el diámetro de la Tierra

# define los controles de las propiedades con los valores por defecto
for n in range(npln):
    t.append(widgets.Text(value=names[n], description='Nombre '+str(n+1), continuous_update=False))
    c.append(widgets.ColorPicker(value=colors[n], description='Color '+str(n+1)))
    s.append(widgets.FloatSlider(value=sizes[n], min=0, max=2, step=0.01, disabled=False, \
                                 description='Tamaño '+str(n+1)))

# agrupa los controles de las propiedades en un solo objeto
con = [numplane]+s+c+t
controls = widgets.VBox(con) 

In [18]:
# based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and 
# https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html

In [19]:
output = widgets.Output() # contiene valores definidos para las propiedades

@output.capture(clear_output=True) # aplica los valores definidos a la función siguiente

# Función para crear la simulación
def runsim(b):    
    global ax, ss
    plt.style.use('dark_background') # fondo negro
    fig = plt.figure(figsize=[6, 6]) # tamaño del cuadro de la animación
    ax = plt.axes([0., 0., 1., 1.], xlim=(-1.8, 1.8), ylim=(-1.8, 1.8)) # definición de los ejes
    ax.set_aspect('equal') # ambos ejes a la misma escala
    ax.axis('off') # ejes invisibles
    # crea un objeto SolarSystem que contiene la simulación
    ss = SolarSystem(Object("Sun", 28, 'yellow', [0, 0, 0], [0, 0, 0])) 
    ss.time = Time(sim_start_date).jd-2 # indica la fecha inicial    
    texty = [.5, .75, 1.05, 1.5] # textos con los nombres
    for i, nasaid in enumerate(range(1, numplane.value+1)): # bucle para clear planetas 
        # consulta posiciones y velocidades iniciales
        obj = Horizons(id=nasaid, location="@sun", epochs=ss.time, id_type='id').vectors() 
        # agrega cada planeta a la simulación
        ss.add_planet(Object(nasaid, 20 * s[i].value, c[i].value, 
                             [np.double(obj[xi]) for xi in ['x', 'y', 'z']],
                             [np.double(obj[vxi]) for vxi in ['vx', 'vy', 'vz']]))
        ax.text(0, - (texty[i] + 0.1), t[i].value, color=c[i].value, zorder=1000, ha='center', fontsize='large')
    # crea animación a partir de imágenes generadas para cada día a simular
    ani = animation.FuncAnimation(fig, animate, repeat=True, frames=sim_duration+1, blit=True, interval=40)
    HTML(ani.to_html5_video()) # crea video de la animación
    rc('animation', html='html5')
    display(ani) # despliega video

In [20]:
# botón para iniciar la ejecución de la simulación
run_button = widgets.Button(description = 'Crear')
run_button.on_click(runsim)

# despliega controles
display(widgets.HBox([controls, output]))
display(run_button)

Button(description='Crear', style=ButtonStyle())